In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [6]:
# Create our features
X = df.drop(columns=target)

# Create our target
y = df[target]

In [7]:
# Creating an instance of label encoder
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# if needed, encode the column keep the schema.
# perform fit_transform on all.
encoded_keys = dict()
for feature in X.columns.values:
    X[feature] = label_encoder.fit_transform(df[feature])
    if ((df.dtypes[feature] != np.int64) and (df.dtypes[feature] != np.float64)):
        encoded_keys[feature] = label_encoder.classes_.tolist()

# trim constant features
trim_features = []
for feature in encoded_keys:
    if len(encoded_keys[feature]) == 1:
        trim_features.append(feature)
for feature in trim_features:
    encoded_keys.pop(feature, None)
X = X.drop(columns=trim_features)

# encoded features
encoded_keys  

{'home_ownership': ['ANY', 'MORTGAGE', 'OWN', 'RENT'],
 'verification_status': ['Not Verified', 'Source Verified', 'Verified'],
 'issue_d': ['Feb-2019', 'Jan-2019', 'Mar-2019'],
 'initial_list_status': ['f', 'w'],
 'next_pymnt_d': ['Apr-2019', 'May-2019'],
 'application_type': ['Individual', 'Joint App']}

In [8]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,dti,delinq_2yrs,inq_last_6mths,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000
mean,572.296903,12.982577,4126.615168,1.812779,3294.087856,0.669994,0.805542,1962.376855,0.217722,0.497697,...,0.052124,2.219408,367.820219,57.069314,0.125972,0.0,27710.412107,24184.046704,295.617696,19966.952715
std,375.779218,9.508701,2367.621789,0.941313,1574.576387,0.719105,0.714932,971.922012,0.717338,0.758122,...,0.389984,1.897307,67.053040,55.878931,0.336732,0.0,17546.728857,14497.653994,257.292302,13661.709989
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,289.000000,5.000000,2153.000000,1.000000,1930.000000,0.000000,0.000000,1255.000000,0.000000,0.000000,...,0.000000,1.000000,347.000000,0.000000,0.000000,0.0,12014.000000,11764.000000,116.000000,7895.000000
50%,516.000000,11.000000,3828.000000,1.000000,3365.000000,1.000000,1.000000,1842.000000,0.000000,0.000000,...,0.000000,2.000000,409.000000,48.000000,0.000000,0.0,26735.000000,23056.000000,221.000000,18429.000000
75%,848.000000,20.000000,5967.000000,3.000000,4665.000000,1.000000,1.000000,2532.000000,0.000000,1.000000,...,0.000000,3.000000,409.000000,101.000000,0.000000,0.0,42861.000000,36149.000000,393.000000,31243.000000
max,1397.000000,51.000000,8809.000000,3.000000,6089.000000,2.000000,2.000000,5695.000000,17.000000,5.000000,...,17.000000,18.000000,409.000000,155.000000,4.000000,0.0,59628.000000,51997.000000,1715.000000,46898.000000


In [9]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [10]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [12]:
# train the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

brf = BalancedRandomForestClassifier(n_estimators=100, random_state=0)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                               criterion='gini', max_depth=None,
                               max_features='auto', max_leaf_nodes=None,
                               max_samples=None, min_impurity_decrease=0.0,
                               min_samples_leaf=2, min_samples_split=2,
                               min_weight_fraction_leaf=0.0, n_estimators=100,
                               n_jobs=None, oob_score=False, random_state=0,
                               replacement=False, sampling_strategy='auto',
                               verbose=0, warm_start=False)

In [13]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7353583551760228

In [14]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

pd.DataFrame(data = confusion_matrix(y_test, y_pred),
            index = ['Actual 0', 'Actual 1'],
            columns = ['Predicted 0', 'Predicted 1'])

,Predicted 0,Predicted 1
Actual 0,61,40
Actual 1,2279,14825


In [15]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.60      0.87      0.05      0.72      0.51       101
   low_risk       1.00      0.87      0.60      0.93      0.72      0.54     17104

avg / total       0.99      0.87      0.61      0.92      0.72      0.54     17205



In [38]:
# List the features sorted in descending order by feature importance
features = {}
for k,v in zip(X_test.columns.to_list(), brf.feature_importances_.tolist()):
    features[k] = v
sorted_features = {}
for k,v in sorted(features.items(), key=lambda x: x[1], reverse=True):
    sorted_features[k] = v
sorted_features

{'total_rec_prncp': 0.08543397206690038,
 'total_rec_int': 0.07136818568247713,
 'total_pymnt': 0.06568416978259736,
 'last_pymnt_amnt': 0.05379888044799111,
 'total_pymnt_inv': 0.053111903670413636,
 'int_rate': 0.037421970106319924,
 'issue_d': 0.02259970745529938,
 'installment': 0.01886517093492613,
 'mths_since_recent_inq': 0.017642401085178987,
 'revol_bal': 0.017347847996297353,
 'max_bal_bc': 0.017070274590828924,
 'out_prncp_inv': 0.01624707158302569,
 'bc_util': 0.01578489313736561,
 'dti': 0.01558638854638658,
 'mths_since_recent_bc': 0.015491197459589602,
 'mo_sin_old_rev_tl_op': 0.014992485908821167,
 'out_prncp': 0.013890671430150527,
 'loan_amnt': 0.013854302343611681,
 'bc_open_to_buy': 0.013830442569666264,
 'mths_since_rcnt_il': 0.01377810739949209,
 'il_util': 0.013744629410676855,
 'annual_inc': 0.013698369912123984,
 'total_bc_limit': 0.013683142059277105,
 'mo_sin_old_il_acct': 0.013541591634597219,
 'total_rev_hi_lim': 0.013519030649028999,
 'all_util': 0.0133938

### Easy Ensemble AdaBoost Classifier

In [41]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier 
eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec.fit(X_train, y_train)

EasyEnsembleClassifier(base_estimator=None, n_estimators=100, n_jobs=None,
                       random_state=1, replacement=False,
                       sampling_strategy='auto', verbose=0, warm_start=False)

In [42]:
# Calculated the balanced accuracy score
y_pred = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7353583551760228

In [43]:
# Display the confusion matrix
pd.DataFrame(data = confusion_matrix(y_test, y_pred),
            index = ['Actual 0', 'Actual 1'],
            columns = ['Predicted 0', 'Predicted 1'])

,Predicted 0,Predicted 1
Actual 0,61,40
Actual 1,2279,14825


In [44]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.60      0.87      0.05      0.72      0.51       101
   low_risk       1.00      0.87      0.60      0.93      0.72      0.54     17104

avg / total       0.99      0.87      0.61      0.92      0.72      0.54     17205

